# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name='./Reacher_Linux_v2/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

T = 8
episodes = 100000
# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
from ddpg_agent20 import Agent, ReplayBuffer
from collections import deque
import torch
scores = np.zeros(num_agents)   # initialize zero-value the score (for each agent)                       

In [11]:
episodes = 1200

reward_list = []

agent = Agent(state_size=state_size, action_size=action_size, random_seed=0)
scores_mean = deque(maxlen=100)
scores = []
best_score = 0
best_average_score = 0
episodes=2000
max_steps=1000
scores_mean = deque(maxlen=100)


for episode in range(1, episodes+1):

    env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
    cur_states = env_info.vector_observations  
    agent.reset()
    scores_agents = np.zeros(num_agents)      
    
    
    for step in range(max_steps):
        actions = agent.act(cur_states)
        env_info = env.step(actions)[brain_name]     
        next_states = env_info.vector_observations   
        rewards = env_info.rewards                   
        dones = env_info.local_done                  
        agent.step(cur_states, actions, rewards, next_states, dones, step)
        cur_states = next_states
        scores_agents += rewards
        if np.any(dones):
            break
    
    score = np.mean(scores_agents)
    scores_mean.append(score)
    average_score = np.mean(scores_mean)
    scores.append(score)
    if score > best_score:
        best_score = score
    if average_score > best_average_score:
        best_average_score = average_score
    print("Episode:{}, Average Score:{:.2f}".format(episode, average_score))
    if average_score > 30:
        torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
        torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
        print("Average score of 30 achieved")            
        break
    
env.close()

/home/f1tor/tf_gpu/lib/python3.6/site-packages/ipykernel_launcher.py:106: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Episode:1, Average Score:0.42
Episode:2, Average Score:0.46
Episode:3, Average Score:0.63
Episode:4, Average Score:0.72
Episode:5, Average Score:0.76
Episode:6, Average Score:0.79
Episode:7, Average Score:0.91
Episode:8, Average Score:1.10
Episode:9, Average Score:1.24
Episode:10, Average Score:1.39
Episode:11, Average Score:1.54
Episode:12, Average Score:1.62
Episode:13, Average Score:1.79
Episode:14, Average Score:2.01
Episode:15, Average Score:2.28
Episode:16, Average Score:2.53
Episode:17, Average Score:2.92
Episode:18, Average Score:3.43
Episode:19, Average Score:4.04
Episode:20, Average Score:4.70
Episode:21, Average Score:5.61
Episode:22, Average Score:6.38
Episode:23, Average Score:7.38
Episode:24, Average Score:8.33
Episode:25, Average Score:9.27
Episode:26, Average Score:10.20
Episode:27, Average Score:11.12
Episode:28, Average Score:12.03
Episode:29, Average Score:12.90
Episode:30, Average Score:13.67
Episode:31, Average Score:14.36
Episode:32, Average Score:15.05
Episode:33

In [6]:
#env.reset()[brain_name].vector_observations[0]
import torch
from ddpg_agent20 import Agent, ReplayBuffer
from collections import deque
from time import sleep

num_agents=20
agent = Agent(state_size=state_size, action_size=action_size, random_seed=0)

agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))


env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
cur_states = env_info.vector_observations  
agent.reset()
scores_agents = np.zeros(num_agents)

for step in range(1100):
    actions = agent.act(cur_states)
    env_info = env.step(actions)[brain_name]     
    next_states = env_info.vector_observations   
    rewards = env_info.rewards                   
    dones = env_info.local_done                  
    agent.step(cur_states, actions, rewards, next_states, dones, step)
    cur_states = next_states
    scores_agents += rewards
    if np.any(dones):
        break

            
env.close()

/home/f1tor/data_nvme/sash/udacity_course/deep-reinforcement-learning/p2_continuous-control/ddpg_agent20.py:127: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(self.critic_local.parameters(), 1)


When finished, you can close the environment.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```